In [1389]:
import tensorflow as tf
from tensorflow import keras
import sys
import numpy as np
fashion_mnist = keras.datasets.fashion_mnist
tf.keras.backend.set_floatx('float64')
np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(suppress=True)

In [1390]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

test_images=np.expand_dims(test_images, axis=3)
train_images=np.expand_dims(train_images, axis=3)
print(train_images.shape)
train_images = train_images /255.0
test_images = test_images /255.0

(60000, 28, 28, 1)


In [1391]:
model = keras.Sequential([
    keras.layers.Conv2D(name='conv1',input_shape=(28,28,1),filters=6, kernel_size=5, strides=1, padding='same',activation='relu'),
    keras.layers.MaxPool2D(name='pool1',pool_size=2),
    keras.layers.Conv2D(name='conv2',filters=16, kernel_size=5, strides=1, padding='valid',activation='relu'),
    keras.layers.MaxPool2D(name='pool2',pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(120, activation='relu',name='fc1'),
    keras.layers.Dense(84, activation='relu',name='fc2'),
    keras.layers.Dense(10,name='fc3'),
    keras.layers.Softmax()
])


In [1392]:
from tensorflow.keras.models import load_model

model.load_weights('initial_weights_alexnet_h5')
print(model.summary())
for l in model.layers:
    print(l.get_config())
    
print(model.layers[5].trainable_variables[0].shape)

Model: "sequential_222"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 28, 28, 6)         156       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 14, 14, 6)         0         
_________________________________________________________________
conv2 (Conv2D)               (None, 10, 10, 16)        2416      
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_190 (Flatten)        (None, 400)               0         
_________________________________________________________________
fc1 (Dense)                  (None, 120)               48120     
_________________________________________________________________
fc2 (Dense)                  (None, 84)             

In [1393]:
from decimal import Decimal
def loss(predictions, label):
    oss=0.0
    loss=predictions[label[0]]
    loss = Decimal(np.log(predictions[label[0]]))
    return loss

predictions = model(train_images[:1])
print(predictions.numpy())
print(train_labels[:1])
print(loss(predictions.numpy()[0], train_labels[:1]))


[[0.10315515 0.0828084  0.089298   0.10259349 0.09754225 0.08758666
  0.1112497  0.10728637 0.09372789 0.12475208]]
[9]
-2.08142688828892996610875343321822583675384521484375


In [1394]:
image_dataset=tf.data.Dataset.from_tensor_slices(train_images[:4]).batch(1)
label_dataset=tf.data.Dataset.from_tensor_slices(train_labels[:4]).batch(1)

loss_object=keras.losses.SparseCategoricalCrossentropy(reduction='sum_over_batch_size')

def loss(model, x, y, training):
    y_ = model(x, training=training)
    return loss_object(y_true=y, y_pred=y_)

def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, training=False)

    return loss_value, tape.gradient(loss_value, model.trainable_variables)

optimizer = keras.optimizers.SGD(learning_rate=0.01)


In [1395]:
input=train_images[:1000]
label=train_labels[:1000]

#newModel2.save_weights('initial_weights_h5',save_format='h5')    
train_loss_results = []
train_accuracy_results = []

num_epochs = 1
newModel = keras.Sequential(model.layers[:1])
for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

    # Training loop - using batches of 32
    for i, (x, y) in enumerate(zip(image_dataset, label_dataset)):
    # Optimize the model
        input=x
        label=y
        loss_value, grads = grad(model, input, label)
        
        y_ = model(input)
        print('model output')
        print(y_)
        print('loss')
        print(loss_value)
        print('filters')
        print(model.layers[0].trainable_variables[1])
        print('grads')

        
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        print(grads[1])
        # Track progress
        #epoch_loss_avg.update_state(loss_value)  # Add current batch loss
        # Compare predicted label to actual label
        # training=True is needed only if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        y_ = model(input)

        #epoch_accuracy.update_state(label, y_)
    

    # End epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

model output
tf.Tensor(
[[0.10315515 0.0828084  0.089298   0.10259349 0.09754225 0.08758666
  0.1112497  0.10728637 0.09372789 0.12475208]], shape=(1, 10), dtype=float64)
loss
tf.Tensor(2.0814268589019775, shape=(), dtype=float64)
filters
<tf.Variable 'conv1_200/bias:0' shape=(6,) dtype=float64, numpy=array([0., 0., 0., 0., 0., 0.])>
grads
tf.Tensor([-0.07368661 -0.18158476  0.11003846 -0.26580147 -0.09740951  0.08516611], shape=(6,), dtype=float64)
model output
tf.Tensor(
[[0.10172734 0.07764438 0.08911259 0.09566537 0.09221403 0.09939929
  0.10701083 0.11388674 0.09024125 0.13309818]], shape=(1, 10), dtype=float64)
loss
tf.Tensor(2.285459041595459, shape=(), dtype=float64)
filters
<tf.Variable 'conv1_200/bias:0' shape=(6,) dtype=float64, numpy=
array([ 0.00073687,  0.00181585, -0.00110038,  0.00265801,  0.0009741 ,
       -0.00085166])>
grads
tf.Tensor([ 0.00906555  0.21478937 -0.09588984  0.03190288 -0.19209093  0.0213418 ], shape=(6,), dtype=float64)
model output
tf.Tensor(
[[0.105